# Importando as biblitecas

In [ ]:
import pandas as pd
import re
import numpy as np

# Lendo o arquivo de mortalidade

In [32]:
Mortalidade_2019 = pd.read_csv(r'C:\Projeto Final\data\Mortalidade_Geral_2019.csv', sep = ';')

C:\Users\Leonardo\AppData\Local\Temp\ipykernel_8720\2560439738.py:1: DtypeWarning: Columns (66) have mixed types. Specify dtype option on import or set low_memory=False.
  Mortalidade_2019 = pd.read_csv(r'C:\Projeto Final\data\Mortalidade_Geral_2019.csv', sep = ';')


# Estudando a tabela 

In [ ]:
Mortalidade_2019.head()

In [ ]:
Mortalidade_2019.info()

In [ ]:
Mortalidade_2019.columns

# Fazendo Verificação e Limpeza da Tabela

In [ ]:
Mortalidade_2019 = Mortalidade_2019.loc[:,['DTOBITO', 'CODMUNOCOR', 'DTNASC', 'IDADE', 'SEXO', 'RACACOR', 'CAUSABAS']]

Vendo os gêneros que mais sofreram com as mortes

In [ ]:
Mortalidade_2019['SEXO'].value_counts()

In [ ]:
#Ao verificar que o Sexo que está igual a 1 possui maior valor, pode-se confirmar que são os homens, pois ao pesquisar sobre a mortalidade de gêneros, os homens são os que mais sofrem mortes

Mudando o formato das datas da coluna de óbito e nascimento

In [ ]:
Mortalidade_2019['DTOBITO'] = Mortalidade_2019['DTOBITO'].astype(str).str.zfill(8)                          # deixa todas as linhas com 8 números, alem de transformar em str
Mortalidade_2019['DTOBITO'] = Mortalidade_2019['DTOBITO'].apply(lambda x: f'{x[0:2]}/{x[2:4]}/{x[4:]}')     # aplica '/' em todas as datas
Mortalidade_2019['DTOBITO'] = pd.to_datetime(Mortalidade_2019['DTOBITO'])                                   # transforma em datetime

In [ ]:
Mortalidade_2019['DTNASC'] = Mortalidade_2019['DTNASC'].fillna(0)                      #transforma nulo em zeros
Mortalidade_2019['DTNASC'] = pd.to_datetime(Mortalidade_2019['DTNASC']).dt.date        #transforma em data
Mortalidade_2019['DTNASC'] = pd.to_datetime(Mortalidade_2019['DTNASC'])

Corrigindo a coluna de idade

In [34]:
Mortalidade_2019['IDADE'] = Mortalidade_2019['IDADE'].astype(str)

def limpeza_idade(linha):
    x = linha['IDADE']
    if linha['DTOBITO']  == linha['DTNASC'] or linha['DTOBITO'] == linha['DTNASC']+1:
        return 0

    elif x.startswith('5') and len(x) == 3:
        x_new_5 = x.replace('5', '1', 1)
        x_new_5 = float(x)
        return x_new_5

    elif x.startswith('4') and len(x) == 3:
        x_new_4 = float(x[1:])
        return x_new_4

    elif x.startswith('3') and len(x) == 3 or len(x) == 2:
        x_new_3 = float(x[1:])
        x_new_3 = (x_new_3 * 833334/100)
        return x_new_3

    elif x.startswith('2') and len(x) == 3 or len(x) == 2:
        x_new_2 = float(x[1:])
        x_new_2 = (x_new_2 * 114155/10000)
        return x_new_2

    elif x.startswith('1') and len(x) == 3 or len(x) == 2:
        x_new_1 = float(x[1:])
        x_new_1 = (x_new_1 * 1.90259/1000000)
        return x_new_1

Mortalidade_2019['IDADE'] = Mortalidade_2019.apply(limpeza_idade, axis = 1)

In [38]:
Mortalidade_2019[Mortalidade_2019['IDADE']<1]

,ORIGEM,TIPOBITO,DTOBITO,HORAOBITO,NATURAL,CODMUNNATU,DTNASC,IDADE,SEXO,RACACOR,...,FONTES,TPRESGINFO,TPNIVELINV,NUDIASINF,DTCADINF,MORTEPARTO,DTCONCASO,FONTESINF,ALTCAUSA,CONTADOR
59,1,2,5022019,1115.0,823.0,230730.0,5022019.0,0.000000,2,4.0,...,SXSXXX,NaN,NaN,NaN,13032019.0,3.0,13032019.0,NaN,2.0,63
86,1,2,13022019,1353.0,833.0,330455.0,12022019.0,0.000038,1,4.0,...,SXXXXX,NaN,NaN,NaN,27052019.0,3.0,27052019.0,NaN,1.0,90
497,1,2,25022019,1516.0,835.0,354780.0,25022019.0,0.000000,1,4.0,...,XXXXXX,NaN,NaN,NaN,27032019.0,9.0,NaN,NaN,NaN,506
623,1,2,24012019,2233.0,835.0,355030.0,24012019.0,0.000000,2,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,636
738,1,2,29012019,1500.0,835.0,350760.0,29012019.0,0.000000,1,1.0,...,SXSXSX,NaN,NaN,NaN,28062019.0,1.0,28062019.0,NaN,2.0,751
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1349751,1,2,19102019,1700.0,813.0,130390.0,19102019.0,0.000000,1,5.0,...,SXXSXX,NaN,NaN,NaN,24122019.0,3.0,20112019.0,NaN,2.0,1378850
1349757,1,2,21022019,600.0,813.0,130140.0,21022019.0,0.000000,1,5.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1378856
1349772,1,2,11092019,1000.0,813.0,130140.0,11092019.0,0.000000,2,5.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1378877
1349774,1,2,27012019,NaN,813.0,130230.0,NaN,0.000000,2,5.0,...,SXXXXX,NaN,NaN,NaN,4062020.0,3.0,4062020.0,NaN,2.0,1378879


### Classificação de causas (coluna CAUSABAS)

    - A00 ~ O99 --> doença

    - P00 ~ Q99 --> problemas na gestão/Parto

    - R00 ~ T98 --> lesões e anomalidades

    - U00 ~ Y98 --> Causas externas de morbidade e de mortalidade (acidente/agressao/intoxicação/falta de comida/falta de agua/queda/tiro)

Arrumando a coluna de Causas para conectar a outra tabela

In [ ]:
Mortalidade_2019['CAUSABAS'] = Mortalidade_2019['CAUSABAS'].apply(lambda x: f'{x[0:3]}')

In [ ]:
Mortalidade_2019